# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [336]:
# Print scikit-learn version
import sklearn
print('sklearn: %s' % sklearn.__version__)

sklearn: 0.20.2


In [337]:
# import libraries
import pandas as pd
import numpy as np
import os
import pickle
from sqlalchemy import create_engine
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score, fbeta_score, classification_report
from scipy.stats import hmean
from scipy.stats.mstats import gmean
import time
import datetime

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\125779\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\125779\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\125779\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\125779\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [338]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table('messages_categories',engine)
X = df['message']
Y = df.iloc[:,3:-2]

In [339]:
# Print category columns
category_cols = df.columns[3:-2].tolist()
print(category_cols)

['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


### 2. Write a tokenization function to process your text data

In [340]:
# Get stop words in 'English' language
stop_words = stopwords.words("english")

In [341]:
# Print length of stop words in English language
print('Length of stop words in English language is {}'.format(len(stop_words)))

Length of stop words in English language is 179


In [342]:
# Print stop words in English language
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [343]:
# Check if any message contain URL link
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
url_count = pd.Series([])
url_count = X.apply(lambda message: len(re.findall(url_regex, message)))
print(type(url_count))
url_count.value_counts().sort_index()    

<class 'pandas.core.series.Series'>


0    25364
1      559
2       78
3       23
4        2
5        1
6        1
Name: message, dtype: int64

From above, we can observe that most of the messages does not have URL links but there are few messages which do contain URL links. There is only 1 observation which contains 5 URL links.

In [344]:
# Define function tokenize to normalize, tokenize and lemmatize text string
def tokenize(text):
    """Normalize, tokenize and lemmatize text string
    
    Args:
    text: string, String containing message for processing
       
    Returns:
    clean_tokens: list, List containing normalized and lemmatized word tokens
    """
    
    # Replace URL links in text string with string 'urlplaceholder'
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    # Substitute characters in text string which match regular expression r'[^a-zA-Z0-9]'
    # with single whitespace
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    
    # Get word tokens from text string
    tokens = word_tokenize(text)
    
    # Instantiate WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()

    # Clean tokens
    clean_tokens = []
    for tok in tokens:
        # convert token to lowercase as stop words are in lowercase
        tok_low = tok.lower() 
        if tok_low not in stop_words:
            # Lemmatize token and remove the leading and trailing spaces from lemmatized token
            clean_tok = lemmatizer.lemmatize(tok_low).lower().strip()
            clean_tokens.append(clean_tok)

    return clean_tokens

In [345]:
# Print first 5 messages and their respective tokens
for idx in X.index.tolist()[0:5]:
    print(X.loc[idx])
    print('-'*100)
    print(tokenize(X.loc[idx]))
    print('*'*100)

Weather update - a cold front from Cuba that could pass over Haiti
----------------------------------------------------------------------------------------------------
['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']
****************************************************************************************************
Is the Hurricane over or is it not over
----------------------------------------------------------------------------------------------------
['hurricane']
****************************************************************************************************
Looking for someone but no name
----------------------------------------------------------------------------------------------------
['looking', 'someone', 'name']
****************************************************************************************************
UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
-------------------------------------

In [346]:
# Print message with url count of 5
print('Index of message with 5 URL links is {}'.format(url_count[url_count == 5].index[0]))
X[url_count[url_count == 5].index[0]]

Index of message with 5 URL links is 12598


'Hurricane Sandy Flight Cancellations: Thousands Of Flights Canceled Due. http://t.co/DMo0tbQE Most read by neighbors in #Roseville #Newarkhappy halloween 2012 (@Frankenstorm Apocalypse - Hurricane Sandy w/ 213 others) http://t.co/DTw9W3kKThe protective cover for the Enterprise failed last night. #NYC #sandy @Space Shuttle Enterprise http://t.co/5jexF6ZG@StormTeam8 @CTPostTrumbull Shelbourne rd Trumbull right next door. Never lost power http://t.co/NqMyZ1H8RT @nytimes: The New York Times is providing free, unlimited access to storm coverage on http://t.co/HkHYUWhW and its mobile apps today.'

In [347]:
# Print message with index location 12598 and its respective tokens
# Index location 12598 contain message with 5 URL links
print(X.loc[12598])
print('-'*100)
print(tokenize(X.loc[12598]))

Hurricane Sandy Flight Cancellations: Thousands Of Flights Canceled Due. http://t.co/DMo0tbQE Most read by neighbors in #Roseville #Newarkhappy halloween 2012 (@Frankenstorm Apocalypse - Hurricane Sandy w/ 213 others) http://t.co/DTw9W3kKThe protective cover for the Enterprise failed last night. #NYC #sandy @Space Shuttle Enterprise http://t.co/5jexF6ZG@StormTeam8 @CTPostTrumbull Shelbourne rd Trumbull right next door. Never lost power http://t.co/NqMyZ1H8RT @nytimes: The New York Times is providing free, unlimited access to storm coverage on http://t.co/HkHYUWhW and its mobile apps today.
----------------------------------------------------------------------------------------------------
['hurricane', 'sandy', 'flight', 'cancellation', 'thousand', 'flight', 'canceled', 'due', 'urlplaceholder', 'read', 'neighbor', 'roseville', 'newarkhappy', 'halloween', '2012', 'frankenstorm', 'apocalypse', 'hurricane', 'sandy', 'w', '213', 'others', 'urlplaceholder', 'protective', 'cover', 'enterpris

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [348]:
# Create a basic pipeline
pipeline_basic = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                           ('tfidf', TfidfTransformer()),
                           ('clf', MultiOutputClassifier(RandomForestClassifier()))
                          ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [349]:
# Split the data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=42)
#start_time = time.time()
start_datetime = datetime.datetime.now().replace(microsecond=0)

# Train basic pipeline
pipeline_basic.fit(X_train, Y_train)
#print("--- %s seconds ---" % (time.time() - start_time))
print("--- Training time: %s ---" % (datetime.datetime.now().replace(microsecond=0) - start_datetime))

--- Training time: 0:01:12 ---


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [350]:
# Predict categories from test set
start_datetime = datetime.datetime.now().replace(microsecond=0)
Y_pred_basic = pipeline_basic.predict(X_test)
print("--- Predicting time: %s ---" % (datetime.datetime.now().replace(microsecond=0) - start_datetime))

--- Predicting time: 0:00:07 ---


In [351]:
# Print type and shape of Y_test and Y_pred
print('Y_test has type: {} and its shape is: {}'.format(type(Y_test), Y_test.shape))
print('Y_pred_basic has type: {} and its shape is: {}'.format(type(Y_pred_basic), Y_pred.shape))

Y_test has type: <class 'pandas.core.frame.DataFrame'> and its shape is: (6507, 35)
Y_pred_basic has type: <class 'numpy.ndarray'> and its shape is: (6507, 35)


In [352]:
# Print first 5 rows of Y_test dataframe
Y_test.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
20861,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12293,1,0,0,1,0,0,0,0,0,1,...,0,0,1,0,1,0,0,0,0,0
25482,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13741,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
14548,1,0,0,1,1,1,1,0,1,0,...,0,1,1,1,1,1,1,1,1,0


In [353]:
# Print first 5 rows in Y_pred ndarray
Y_pred_basic[0:5]

array([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1],
       [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0]], dtype=int64)

In [354]:
# Print accuracy of basic pipeline for each of individual category 
accuracy_basic = (Y_pred_basic == Y_test).mean()
accuracy_basic

related                   0.809897
request                   0.885508
offer                     0.996004
aid_related               0.756109
medical_help              0.919625
medical_products          0.948978
search_and_rescue         0.971415
security                  0.979099
military                  0.967573
water                     0.948210
food                      0.940526
shelter                   0.932688
clothing                  0.985400
money                     0.974950
missing_people            0.989396
refugees                  0.966959
death                     0.958199
other_aid                 0.869679
infrastructure_related    0.934532
transport                 0.956047
buildings                 0.948671
electricity               0.980790
tools                     0.993853
hospitals                 0.989089
shops                     0.994621
aid_centers               0.988935
other_infrastructure      0.956662
weather_related           0.863839
floods              

In [355]:
# Print overall accuracy of basic pipeline
overall_accuracy_basic = (Y_pred_basic == Y_test).mean().mean()
print('Overall accuracy of basic pipeline is: {}%'.format(round(overall_accuracy_basic*100, 2)))

Overall accuracy of basic pipeline is: 94.29%


In [356]:
# Define function to calculate the multi-label f-score
def multi_label_fscore(y_true, y_pred, beta=1):
    """Calculate individual weighted average fbeta score of each category and
    geometric mean of weighted average fbeta score of each category
    
    Args:
    y_true: dataframe, dataframe containing true labels i.e. Y_test
    y_pred: ndarray, ndarray containing predicted labels i.e. Y_pred
    beta: numeric, beta value
       
    Returns:
    f_score_gmean: float, geometric mean of fbeta score for each category
    """
    
    b = beta
    f_score_dict = {}
    score_list = []
    
    # Create dataframe y_pred_df from ndarray y_pred 
    y_pred_df = pd.DataFrame(y_pred, columns=y_true.columns)

    for column in y_true.columns:
        score = round(fbeta_score(y_true[column], y_pred_df[column], beta, average='weighted'),4)
        score_list.append(score)
    f_score_dict['category'] = y_true.columns.tolist()
    f_score_dict['f_score'] = score_list

   
    f_score_df = pd.DataFrame.from_dict(f_score_dict)
#    print(f_score_df)

    f_score_gmean = gmean(f_score_df['f_score'])

    return f_score_gmean

In [357]:
# Print overall f_score of basic pipeline
multi_f_gmean_basic = multi_label_fscore(Y_test,Y_pred_basic, beta = 1)
print('Overall F_beta_score for basic pipeline is: {0:.2f}%'.format(multi_f_gmean_basic*100))

Overall F_beta_score for basic pipeline is: 92.81%


C:\Users\125779\AppData\Local\Continuum\anaconda3\envs\dsnd\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [358]:
# Report the basic pipeline f1 score, precision and recall for each output category of the dataset
# by iterating through the columns and calling sklearn's classification_report on each column
for column in Y_test.columns:
    print('------------------------------------------------------\n')
    print('CATEGORY: {}\n'.format(column))
    print(classification_report(Y_test[column],pd.DataFrame(Y_pred_basic, columns=Y_test.columns)[column]))

------------------------------------------------------

CATEGORY: related

              precision    recall  f1-score   support

           0       0.66      0.45      0.53      1569
           1       0.84      0.93      0.88      4938

   micro avg       0.81      0.81      0.81      6507
   macro avg       0.75      0.69      0.71      6507
weighted avg       0.80      0.81      0.80      6507

------------------------------------------------------

CATEGORY: request

              precision    recall  f1-score   support

           0       0.90      0.97      0.93      5403
           1       0.78      0.45      0.57      1104

   micro avg       0.89      0.89      0.89      6507
   macro avg       0.84      0.71      0.75      6507
weighted avg       0.88      0.89      0.87      6507

------------------------------------------------------

CATEGORY: offer

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6481
           1  

C:\Users\125779\AppData\Local\Continuum\anaconda3\envs\dsnd\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



------------------------------------------------------

CATEGORY: military

              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6282
           1       0.69      0.11      0.19       225

   micro avg       0.97      0.97      0.97      6507
   macro avg       0.83      0.55      0.59      6507
weighted avg       0.96      0.97      0.96      6507

------------------------------------------------------

CATEGORY: water

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6077
           1       0.83      0.27      0.41       430

   micro avg       0.95      0.95      0.95      6507
   macro avg       0.89      0.63      0.69      6507
weighted avg       0.94      0.95      0.94      6507

------------------------------------------------------

CATEGORY: food

              precision    recall  f1-score   support

           0       0.95      0.98      0.97      5776
           1   

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6435
           1       0.33      0.01      0.03        72

   micro avg       0.99      0.99      0.99      6507
   macro avg       0.66      0.51      0.51      6507
weighted avg       0.98      0.99      0.98      6507

------------------------------------------------------

CATEGORY: earthquake

              precision    recall  f1-score   support

           0       0.97      0.99      0.98      5901
           1       0.89      0.70      0.78       606

   micro avg       0.96      0.96      0.96      6507
   macro avg       0.93      0.84      0.88      6507
weighted avg       0.96      0.96      0.96      6507

------------------------------------------------------

CATEGORY: cold

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6367
           1       0.86      0.09      0.16       140

   micro avg       0.98      0.9

In [359]:
# Create dict for classification report containg metrics for each of the label for basic pipeline
clf_report_dict_basic = {}
for column in Y_test.columns:
    clf_report_dict_basic[column] = classification_report(Y_test[column],\
                                                          pd.DataFrame(Y_pred_basic, columns=Y_test.columns)[column],\
                                                          output_dict=True)
clf_report_dict_basic

{'related': {'0': {'precision': 0.6551401869158878,
   'recall': 0.4467813894200127,
   'f1-score': 0.5312618416066691,
   'support': 1569},
  '1': {'precision': 0.8403531359205444,
   'recall': 0.925273390036452,
   'f1-score': 0.8807710843373495,
   'support': 4938},
  'micro avg': {'precision': 0.809897033963424,
   'recall': 0.809897033963424,
   'f1-score': 0.8098970339634242,
   'support': 6507},
  'macro avg': {'precision': 0.7477466614182161,
   'recall': 0.6860273897282324,
   'f1-score': 0.7060164629720094,
   'support': 6507},
  'weighted avg': {'precision': 0.7956936742656642,
   'recall': 0.809897033963424,
   'f1-score': 0.7964956883262173,
   'support': 6507}},
 'request': {'0': {'precision': 0.8967632027257241,
   'recall': 0.9742735517305201,
   'f1-score': 0.9339128892042935,
   'support': 5403},
  '1': {'precision': 0.7817896389324961,
   'recall': 0.45108695652173914,
   'f1-score': 0.572085008615738,
   'support': 1104},
  'micro avg': {'precision': 0.8855079145535

In [360]:
# Define function to create dataframe containing only weighted avg(precision, recall & f1-score) for each of the label
# from the classification report dict

def weighted_avg_metric(clf_report_dict):
    """Create dataframe containing only weighted avg(precision, recall & f1-score) for each of the label
    from the classification report dict
    
    Args:
    classification report: dict, dict containing classification report for each of the label
       
    Returns:
    weighted avg metrics: dataframe, dataframe containing weighted avg metrics(precision, recall & f1-score)
    for each of the label
    """
        
    clf_idx = []
    clf_metric_precision = []
    clf_metric_recall = []
    clf_metric_f1_score = []
    metric_dict = {}
    for key,value in clf_report_dict.items():
        clf_idx.append(key)
        clf_metric_precision.append(value['weighted avg']['precision'])
        clf_metric_recall.append(value['weighted avg']['recall'])
        clf_metric_f1_score.append(value['weighted avg']['f1-score'])

    metric_dict['precision'] = clf_metric_precision
    metric_dict['recall'] = clf_metric_recall
    metric_dict['f1_score'] = clf_metric_f1_score

    clf_report_df = pd.DataFrame(metric_dict, index=clf_idx)
        
    return clf_report_df

In [361]:
# Calculate weighted avg metric for basic pipeline and concatenate accuracy calculated above for each
# of the label to form a new dataframe
final_metric_basic = pd.concat([weighted_avg_metric(clf_report_dict_basic), accuracy_basic], axis=1)
#print(final_metric_basic.columns)
final_metric_basic.rename(columns={0:'accuracy'}, inplace=True)
final_metric_basic

,precision,recall,f1_score,accuracy
related,0.795694,0.809897,0.796496,0.809897
request,0.877256,0.885508,0.872524,0.885508
offer,0.992025,0.996004,0.994010,0.996004
aid_related,0.756016,0.756109,0.751185,0.756109
medical_help,0.900253,0.919625,0.888892,0.919625
medical_products,0.937178,0.948978,0.929009,0.948978
search_and_rescue,0.962195,0.971415,0.960944,0.971415
security,0.963762,0.979099,0.969510,0.979099
military,0.959596,0.967573,0.956073,0.967573
water,0.942834,0.948210,0.935705,0.948210


In [363]:
# Print overall weighted avg accuracy for basic pipeline
gmean(final_metric_basic['accuracy'])

0.9411793317772238

In [362]:
# Print overall weighted avg f1_score for basic pipeline
gmean(final_metric_basic['f1_score'])

0.9281140621482137

### 6. Improve your model
Use grid search to find better parameters. 

#### 6.1 Add custom Estimator

In [364]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
#        print('*'*100)
#        print(sentence_list)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
#            print(pos_tags)
#            print(type(pos_tags))
            if pos_tags:
                first_word, first_tag = pos_tags[0]
                if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                    return float(True)
        return float(False)

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [365]:
type(X_train[:5])

pandas.core.series.Series

In [366]:
type(pd.Series(X_train[:5]))

pandas.core.series.Series

In [367]:
for idx in X_train.index[0:5]:
    print(idx, X_train[idx])

2796 Petit Goave #1 needs food. Where can we sleep. Please, we're asking not take years because we can't survive?
16642 Monsoon downpours and upstream waters cascading down the hills from across the border in northeastern India, swamped towns and villages in Bangladesh inundating mud walled dwellings, rice farms, roads, bridges and factories.
17486 Cyclone Nargis hit the densely populated delta last month, killing up to 134,000 people and leaving 2.4 million destitute.
4034 I received all your SMS but I never got any aid where I live. Thank you. 
25729 In the South-South states of Delta and Rivers, VAWG incidents often occur within the context of domestic, criminal and cult violence; with women being both targets as well as innocent bystanders swept up in the general insecurity.


In [368]:
SVE = StartingVerbExtractor()
#SVE.fit(X_train[:5]).tranform(X_train)
dir(SVE)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_get_param_names',
 'fit',
 'fit_transform',
 'get_params',
 'set_params',
 'starting_verb',
 'transform']

In [369]:
SVE = StartingVerbExtractor()
SVE.starting_verb("Petit Goave #1 needs food. Where can we sleep. Please, we're asking not take years because we can't survive?")

1.0

#### 6.2 Improve pipeline

In [370]:
# Create a new improved pipeline
pipeline_new = Pipeline([('features', FeatureUnion([
                                        ('text_pipeline', Pipeline([
                                            ('vect', CountVectorizer(tokenizer=tokenize)),
                                            ('tfidf', TfidfTransformer())])),
                                        ('starting_verb', StartingVerbExtractor())])),
                         ('clf', MultiOutputClassifier(RandomForestClassifier()))
                    ])

#### 6.3 Specify parameters for grid search

In [371]:
# Specify parameters for grid search
parameters = {
    'features__text_pipeline__vect__ngram_range': [(1,2)],
    'features__text_pipeline__vect__max_df': [0.75],
    'features__text_pipeline__vect__max_features': [5000],
    'features__text_pipeline__tfidf__use_idf': [True],
#   'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
#   'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
#   'features__text_pipeline__vect__max_features': (None, 5000, 10000),
#   'features__text_pipeline__tfidf__use_idf': (True, False),
    'clf__estimator__n_estimators': [200],
    'clf__estimator__min_samples_split': [4],
#    'clf__estimator__n_estimators': [50,100,200],
#    'clf__estimator__min_samples_split': [2,3,4],
    'features__transformer_weights': (
        {'text_pipeline': 1, 'starting_verb': 0.5},
#        {'text_pipeline': 0.5, 'starting_verb': 1},
#        {'text_pipeline': 0.8, 'starting_verb': 1},
    )
}

#### 6.3 Define custom scorer

In [372]:
# Specify custom scorer
scorer = make_scorer(multi_label_fscore,greater_is_better = True)

#### 6.4 Create grid search object

In [373]:
# create grid search object
cv = GridSearchCV(pipeline_new, param_grid=parameters, scoring=scorer,verbose = 2)

In [374]:
# Print grid search CV object params
from pprint import pprint
import json

#data = json.dumps(my_dict, indent=1)
#data=pipeline_new.get_params().keys()
pprint(pipeline_new.get_params())
#type(pipeline_new.get_params())

{'clf': MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None),
 'clf__estimator': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
 'clf__estimator__bootstrap': True,
 'clf__estimator__clas

In [375]:
# Fit GridSearchCV object to training set
cv.fit(X_train, Y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=200, features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__max_df=0.75, features__text_pipeline__vect__max_features=5000, features__text_pipeline__vect__ngram_range=(1, 2), features__transformer_weights={'text_pipeline': 1, 'starting_verb': 0.5} 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\125779\AppData\Local\Continuum\anaconda3\envs\dsnd\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=200, features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__max_df=0.75, features__text_pipeline__vect__max_features=5000, features__text_pipeline__vect__ngram_range=(1, 2), features__transformer_weights={'text_pipeline': 1, 'starting_verb': 0.5}, total= 9.2min
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=200, features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__max_df=0.75, features__text_pipeline__vect__max_features=5000, features__text_pipeline__vect__ngram_range=(1, 2), features__transformer_weights={'text_pipeline': 1, 'starting_verb': 0.5} 

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 11.3min remaining:    0.0s
C:\Users\125779\AppData\Local\Continuum\anaconda3\envs\dsnd\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=200, features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__max_df=0.75, features__text_pipeline__vect__max_features=5000, features__text_pipeline__vect__ngram_range=(1, 2), features__transformer_weights={'text_pipeline': 1, 'starting_verb': 0.5}, total= 8.2min
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=200, features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__max_df=0.75, features__text_pipeline__vect__max_features=5000, features__text_pipeline__vect__ngram_range=(1, 2), features__transformer_weights={'text_pipeline': 1, 'starting_verb': 0.5} 


C:\Users\125779\AppData\Local\Continuum\anaconda3\envs\dsnd\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=200, features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__max_df=0.75, features__text_pipeline__vect__max_features=5000, features__text_pipeline__vect__ngram_range=(1, 2), features__transformer_weights={'text_pipeline': 1, 'starting_verb': 0.5}, total= 7.5min


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 30.7min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=None,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, ma..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'features__text_pipeline__vect__ngram_range': [(1, 2)], 'features__text_pipeline__vect__max_df': [0.75], 'features__text_pipeline__vect__max_features': [5000], 'features__text_pipeline__tfidf__use_idf': [True], 'clf__estimator__n_estimators': [200], 'clf__estimator__min_samples_split': [4], 'features__transformer_weights': ({'text_pipeline': 1, 'starting_verb': 0.5},)},
       pre_dispatch='2*n_jobs', refit=True, return_t

**After trying various combinations of parameters defined in parameter's grid, we have the best estimator as per the metrics on the training set and below are the details for best estimator and parameters.**

In [376]:
# Print best estimator for GridSearchCV object
cv.best_estimator_

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=None,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, ma..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))])

In [377]:
# Print best estimator's all parameters
pprint(cv.best_estimator_.get_params())

{'clf': MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=4,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None),
 'clf__estimator': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=4,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
 'clf__estimator__bootstrap': True,
 'clf__estimator__class_weig

In [378]:
# Print the score(as per custom scorer) on the training set after the best estimator selected has been refit 
cv.best_score_

0.9362128452922311

In [379]:
# Print best estimator parameters selected from the parameter's grid with their values 
cv.best_params_

{'clf__estimator__min_samples_split': 4,
 'clf__estimator__n_estimators': 200,
 'features__text_pipeline__tfidf__use_idf': True,
 'features__text_pipeline__vect__max_df': 0.75,
 'features__text_pipeline__vect__max_features': 5000,
 'features__text_pipeline__vect__ngram_range': (1, 2),
 'features__transformer_weights': {'text_pipeline': 1, 'starting_verb': 0.5}}

In [380]:
# Print scorer used to select the best parameters for the GridSearchCV object
cv.scorer_

make_scorer(multi_label_fscore)

In [381]:
# Print the number of cross-validation splits (folds/iterations) used while fitting the training set
cv.n_splits_

3

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [382]:
# Predict categories from test set using tuned pipeline
start_datetime = datetime.datetime.now().replace(microsecond=0)
Y_pred_tuned = cv.predict(X_test)
print("--- Predicting time: %s ---" % (datetime.datetime.now().replace(microsecond=0) - start_datetime))

--- Predicting time: 0:01:09 ---


In [383]:
# Print accuracy of tuned pipeline for each of individual category 
accuracy_tuned = (Y_pred_tuned == Y_test).mean()
accuracy_tuned

related                   0.822038
request                   0.895651
offer                     0.996004
aid_related               0.776702
medical_help              0.920240
medical_products          0.955125
search_and_rescue         0.971723
security                  0.979099
military                  0.967420
water                     0.964961
food                      0.952359
shelter                   0.946058
clothing                  0.987552
money                     0.975411
missing_people            0.989703
refugees                  0.968803
death                     0.967266
other_aid                 0.875211
infrastructure_related    0.934839
transport                 0.957123
buildings                 0.954972
electricity               0.981251
tools                     0.993853
hospitals                 0.989089
shops                     0.994621
aid_centers               0.988935
other_infrastructure      0.956355
weather_related           0.891655
floods              

In [384]:
# Print overall accuracy of tuned pipeline
overall_accuracy_tuned = (Y_pred_tuned == Y_test).mean().mean()
print('Overall accuracy of tuned pipeline is: {}%'.format(round(overall_accuracy_tuned*100, 2)))

Overall accuracy of tuned pipeline is: 94.88%


In [385]:
# Print overall f_score of tuned pipeline
multi_f_gmean_tuned = multi_label_fscore(Y_test,Y_pred_tuned, beta = 1)
print('Overall F_beta_score of tuned pipeline is: {0:.2f}%'.format(multi_f_gmean_tuned*100))

C:\Users\125779\AppData\Local\Continuum\anaconda3\envs\dsnd\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Overall F_beta_score of tuned pipeline is: 93.73%


In [387]:
# Report the tuned pipeline f1 score, precision and recall for each output category of the dataset
# by iterating through the columns and calling sklearn's classification_report on each column
for column in Y_test.columns:
    print('------------------------------------------------------\n')
    print('CATEGORY: {}\n'.format(column))
    print(classification_report(Y_test[column],pd.DataFrame(Y_pred_tuned, columns=Y_test.columns)[column]))

------------------------------------------------------

CATEGORY: related

              precision    recall  f1-score   support

           0       0.70      0.46      0.55      1569
           1       0.84      0.94      0.89      4938

   micro avg       0.82      0.82      0.82      6507
   macro avg       0.77      0.70      0.72      6507
weighted avg       0.81      0.82      0.81      6507

------------------------------------------------------

CATEGORY: request

              precision    recall  f1-score   support

           0       0.91      0.97      0.94      5403
           1       0.79      0.52      0.63      1104

   micro avg       0.90      0.90      0.90      6507
   macro avg       0.85      0.75      0.78      6507
weighted avg       0.89      0.90      0.89      6507

------------------------------------------------------

CATEGORY: offer

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6481
           1  

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6380
           1       0.63      0.09      0.16       127

   micro avg       0.98      0.98      0.98      6507
   macro avg       0.81      0.55      0.58      6507
weighted avg       0.98      0.98      0.97      6507

------------------------------------------------------

CATEGORY: tools

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6467
           1       0.00      0.00      0.00        40

   micro avg       0.99      0.99      0.99      6507
   macro avg       0.50      0.50      0.50      6507
weighted avg       0.99      0.99      0.99      6507

------------------------------------------------------

CATEGORY: hospitals

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6436
           1       0.00      0.00      0.00        71

   micro avg       0.99      0.9

In [390]:
# Create dict for classification report containg metrics for each of the label for tuned pipeline
clf_report_dict_tuned = {}
for column in Y_test.columns:
    clf_report_dict_tuned[column] = classification_report(Y_test[column],\
                                                          pd.DataFrame(Y_pred_tuned, columns=Y_test.columns)[column],\
                                                          output_dict=True)
clf_report_dict_tuned

{'related': {'0': {'precision': 0.7020648967551623,
   'recall': 0.45506692160611856,
   'f1-score': 0.5522041763341067,
   'support': 1569},
  '1': {'precision': 0.8442622950819673,
   'recall': 0.9386391251518833,
   'f1-score': 0.8889528193325661,
   'support': 4938},
  'micro avg': {'precision': 0.822037805440295,
   'recall': 0.822037805440295,
   'f1-score': 0.822037805440295,
   'support': 6507},
  'macro avg': {'precision': 0.7731635959185648,
   'recall': 0.6968530233790009,
   'f1-score': 0.7205784978333364,
   'support': 6507},
  'weighted avg': {'precision': 0.8099749556052872,
   'recall': 0.822037805440295,
   'f1-score': 0.8077543221964691,
   'support': 6507}},
 'request': {'0': {'precision': 0.9082267542343588,
   'recall': 0.9726078104756617,
   'f1-score': 0.9393153990526409,
   'support': 5403},
  '1': {'precision': 0.7947295423023578,
   'recall': 0.5190217391304348,
   'f1-score': 0.6279452054794521,
   'support': 1104},
  'micro avg': {'precision': 0.895650837559

In [391]:
# Calculate weighted avg metric for tuned pipeline and concatenate accuracy calculated above for each
# of the label to form a new dataframe
final_metric_tuned = pd.concat([weighted_avg_metric(clf_report_dict_tuned), accuracy_tuned], axis=1)
final_metric_tuned.rename(columns={0:'accuracy'}, inplace=True)
final_metric_tuned

,precision,recall,f1_score,accuracy
related,0.809975,0.822038,0.807754,0.822038
request,0.888970,0.895651,0.886487,0.895651
offer,0.992025,0.996004,0.994010,0.996004
aid_related,0.775698,0.776702,0.775953,0.776702
medical_help,0.899825,0.920240,0.892857,0.920240
medical_products,0.948432,0.955125,0.942938,0.955125
search_and_rescue,0.963398,0.971723,0.961364,0.971723
security,0.963762,0.979099,0.969510,0.979099
military,0.958771,0.967420,0.956180,0.967420
water,0.962268,0.964961,0.962080,0.964961


In [392]:
# Print overall weighted avg accuracy for tuned pipeline
gmean(final_metric_tuned['accuracy'])

0.9473721335768371

In [393]:
# Print overall weighted avg f1_score for tuned pipeline
gmean(final_metric_tuned['f1_score'])

0.9373304418224195

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

So, from above, we can observe that after tuning the basic pipeline, tuned pipeline was able to perform slightly better in terms of both accuracy and f1-score.

* **Accuracy of basic pipeline was 94.29% whereas accuracy of tuned pipeline is 94.88%.**
* **Also, f1-score of basic pipleine was 92.81% whereas f1-score of tuned pipeline is 93.73%**

Below are the changes which we introduced while tuning the basic pipeline in order to have tuned pipleine.

* **Added another feature 'starting_verb' by using custom estimator 'StartingVerbExtractor' and performed Feature Union along with TF-IDF.**
* **Tuned various parameters of estimators(transformers & classifier) in order to have the best estimator.**
* **The best estimator parameters are:**

{'clf__estimator__min_samples_split': 4,  
 'clf__estimator__n_estimators': 200,   
 'features__text_pipeline__tfidf__use_idf': True,  
 'features__text_pipeline__vect__max_df': 0.75,  
 'features__text_pipeline__vect__max_features': 5000,  
 'features__text_pipeline__vect__ngram_range': (1, 2),  
 'features__transformer_weights': {'text_pipeline': 1, 'starting_verb': 0.5}}
 
We can try further impriving the perfromance metrics(accuracy & f1-score) by using other classifiers like AdaBoost or SVM. But since, this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass, therefore, we would proceed further by taking tuned pipeline into account.

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.